# Проект "OCR-справка"

## 1. Задание

Для облегчения работы модераторов Заказчика требуется разработать решение для распознавания изображений (фотографий) справок о донациях, оформленных по форме 405. Необходимо рассмотреть и выбрать одно из существующих решений для обнаружения и обработки отсканированных таблиц или разработать свой подход. Оценить качество этого решения на данных, предоставленных Заказчиком.

Результат работы:
- командный репозиторий с файлами (в readme.md описание задачи и краткиq итог работы);
- отчет о ходе экспериметров и выбранном подходе;
- микросервис для интеграции в систему Заказчика.

На изображении справки требуется распознать:
- полная дата донации (число, месяц, год в формате дд.мм.гггг);
- тип донации (платно или безвозмездно);
- вид донации (цельная кровь, плазма, тромбоциты, эритроциты, гранулоциты (лейкоциты));
- дополнительно:
    - центр крови;
    - город.

Условные обозначения и сокращения:
- кр/д - кроводача (то есть цельная кровь);
- пл/д - плазмадача (плазма);
- ц/д - цитосдача (тромбоциты);
- БВ - безвозмездно;
- ПЛАТ - платно.

## 2. Проработка решения

В качестве вариантов решения рассмотрено две гипотезы:
1. Можно реализовать проект с использованием существующих решений по разпознаванию таблиц.
2. Необходима разработка собственного решения по распознаванию таблиц.

Преимуществами первой гипотезы являются простота и скорость развертывания, преимуществом второй гипотезы является полная контролируемость и управляемость.

С учетом поставленных сроков и опыта команды выбрано направление по подбору и использованию существующих решений.

### 2.1 Проверка гипотезы использования существующих решений

#### 2.1.1 Использование библиотеки `pytesseract`

Достоинства библиотеки: Поддержка русского языка, актуальность, популярность, работа "из коробки", возможность сохранения результата в pandas-датафрейм.

Результата рассмотрения: Текстовая информация распознается с ошибками, даже таблицы хорошего качества не распознаются. Применение нецелесообразно.

#### 2.1.2 Использование библиотеки `img2table`

Достоинства библиотеки: Библиотека является оболочкой для развертывания систем популярных OCR-решений. Имеет направленность на распознавание таблиц и возможность сохранения результата распознавания таблиц в pandas-датафрейме и xls или csv файле.

Поддерживаемые в `img2table` OCR-решения:
- Tesseract;
- PaddleOCR;
- EasyOCR;
- Google Vision;
- AWS Textract;
- Azure Cognitive Services.

Результаты рассмотрения:
- Tesseract. Тестовая таблица (крупные ячейка, крупные четкие цифры и буквы) распознается с ошибками. Таблицы на всех примерах справок (даже имеющих хорошее качество) не распознаются.
- PaddleOCR. Тестовая таблица распознается с ошибками, таблицы со справок не распознаются.
- EasyOCR. Тестовая таблица распознается без ошибок, таблицы со справок распознаются с большими ошибками, только со справок хорошего качества.
- Google Vision, AWS Textract, Azure Cognitive Services. Все сервисы требуют регистрации и ключа аутентификации, доступны тарифные планы с некоторыми ограничениями, регистрация недоступка на жителей России (в связи с санкциями). Тестирование не проводилось.
- Для улучшения распознавания применялось предварительное вырезание таблиц из справок и простая предобработка изображений срествами библиотеки `OpenCV`. Предварительное вырезание таблиц приводит к улучшению распознавания, предобработка срествами библиотеки `OpenCV` положительного эффекта не дает.

#### 2.1.3 Вывод по результатам рассмотрения первой гипотезы (использование существующих решений)

В результате тестирования двух существующих "коробочных" решений и применения нескольких OCR (в случае использования `img2table`) получить примелемое или близкое к примелемому качество распознавания таблиц не удалось. 

В целом в сегменте OCR-продуктов подбор решений, необходимых для реализации данного проекта, ограничивается (субъективно, в порядке уменьшения влияния):
- платностью или необходимостью регистрации;
- отсутствием API (сервисы он-лайн распознавания);
- отсутствием полноценной поддержки распознавания таблиц;
- отсутствием необходимого языкового пакета.

По результатам рассмотрения и тестирования данная гипотеза отвергнута.

### 2.2 Проверка гипотезы разработки собственного решения

Понимая нецелесообразность 100% написания всего кода "с нуля", допускаем при написании кода использование стандартных общепринятых открытых библиотек.

Учитывая цели задания и опыт рассмотрения "коробочных" OCR-решений, определим следующую структуру разрабатываемого кода:
1. Загрузка изображений/фото из файлов и их предобработка.
2. Обнаружение и извлечение таблицы из изображения.
3. Распознавание содержимого ячеек и сохранение результата.
4. Сравнение результата распознавания с проверочными csv-файлами, расчет метрики.
5. Создание микросервиса для включения в систему Зказчика.



#### 2.2.1 Загрузка изображений/фото из файлов и их предобработка

Загрузка изображений реализована стандартными средствами библиотеки `Pillow` из папки с заранее сохраненными изображениями справок.

Большая часть предоставленных Заказчиком изображений/фото имеет низкое или очень плохое качество, что негативно влияет на результаты распознавания. В связи с этим было принято решение ввести этап предобработки, рассмотрены и протестированы следующие варианты:
- Предобработка с помощью функций и методов библиотек `OpenCV` (перевод в градации серого, инвертирование, двусторонняя фильтрация, адаптивный порог и т.п.).
- Использование библиотеки `jdeskew` для выравнивания изображений.
- Предобученные модели:
  - `MIRNet` для улучшения освещения на изображениях;
  - `H-DIBCO 2018` для бинаризации изображений;
  - `RRDN` для увеличения точности.
- Разработана функция для увеличения точности изображения с использованием предобученной модели `RealESRGAN`.

Большая часть методов не дала положительного эффекта и приводила к ухудшению распознавания. Самым эффективным стало применение адаптивного порога к изображению средствами библиотеки `OpenCV`.

Использование предобученной модели `RealESRGAN` дает положительный эффект для плохих изображений и отрицательный эффект для хороших изображений, поэтому данную предобработку решено добавить в состав микросервиса для запуска пользователем в ручном режиме в случае получения некачественного распознавания.

#### 2.2.2 Обнаружение и извлечение таблицы из изображения

Разработана функция для обнаружения и вырезания таблицы с использованием библиотек `Pillow` и `OpenCV`, которая после применения адаптивного порога находит на изображении границы контуров и вырезает из них самый большой (так как таблица на справке у нас всегда одна).

#### 2.2.3 Распознавание содержимого ячеек и сохранение результата

Пробовали тестировать предобученные модели: CRNN, Keras OCR, Rosetta, RCNN и MobileNet. Не получилось из-за ошибок при загрузке.

Пришли к необходимости создания пользовательской функции, которая для распознавания подключает OCR-библиотеки. Эксперементировали с сочетанием библиотек для разных фрагментов/ячеек таблицы. Лучшее качество распознавания (при не самом лучшем времени распознавания) получилось при сочетании `pytesseract` и `EasyOCR`, это сочетание включено в итоговый вариант функции распознавания.

Пользовательская функция распознавания работает на вырезанном из справки изображении таблицы, определяя границы и контура ячеек. Затем содержимое каждой ячейки распознается с помощью "быстрой" библиотеки `pytesseract`, которая лучше распознает текст. В случае неудачного распознавания или обраружения в ячейке цифр, содержимое ячейки перераспознается в помощью более медленной, но более качественно распознающей цифры библиотеки `EasyOCR`.

Сохранение результата распознавания осуществляется в датасет из 4 столбцов: Дата; Тип донации; Вид донации; Количество.  
Распознавание происходит построчно и поячеечно с ориентацией на дату распознавания, как ключевое поле для формирования записи о донации.

#### 2.2.4 Сравнение результата распознавания с проверочными csv-файлами, расчет метрики

Для оценки качества распознавания проведено тестирование на 15 изображениях справок, предоставленных Заказчиком и имеющих проверочные csv-файлы для оценки результата.

В зависимости от качества исходных изображений метрика (процент корректно распознанных ячеек) составила от 0 до 100%.

Среднее значение метрики для всего набора из 15 изображений составило 8,67%.

Основное отрицательное влияние на результат распознаания оказывает низкое качество изображений (фото) и наклон таблиц.

In [ ]:
# импорт библиотек
import re
import os
import time
import datetime
import cv2
import numpy as np
import pandas as pd
import pytesseract
import easyocr
import dateparser
import imutils
from PIL import Image
from ultralyticsplus import YOLO
from IPython.display import display

##### Функции

###### Функция вырезания таблицы из изображения `crop_img`

In [ ]:
# функция для вырезания таблицы из изображения
def crop_img(img_path, output_path):
    # Считывание изображения
    img = Image.open(img_path)
    array = np.array(img)
    try:
        results = model.predict(img)

        for result in results:
            bbox = result.boxes.xyxy[0].tolist()
            x_min, y_min, x_max, y_max = bbox
            reserve = int(array.shape[0] / array.shape[1] * (x_max / x_min))
            reserve2 = int(array.shape[0] / array.shape[1] * (y_max / x_min))
            cropped_image = img.crop((int(x_min) - reserve,
                                      int(y_min) - reserve,
                                      int(x_max) + reserve + reserve2,
                                      int(y_max) + reserve + reserve2))
            cropped_image = np.array(cropped_image)
    except:
        print(1)
        if array.ndim == 3 and array.shape[2] == 3:
            array = cv2.cvtColor(array, cv2.COLOR_BGR2GRAY)
        # Присвоение изображению порогового значения в виде двоичного изображения
        img_bin = cv2.adaptiveThreshold(array, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

        # Обнаружение контуров
        contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        try:
            new_contours = []
            for contour in contours:
                _, _, w, h = cv2.boundingRect(contour)
                if 0.6 < w / array.shape[1] < 1 and 0 < h / array.shape[0] < 0.4:
                    new_contours.append(contour)

            # Определение самого большого контура
            largest_contour = max(new_contours, key=cv2.contourArea)
        except:
            largest_contour = max(contours, key=cv2.contourArea)

        # Обрезание по этому контуру
        x, y, w, h = cv2.boundingRect(largest_contour)
        cropped_image = array[y:y + h, x:x + w]
    
    # Сохранение вырезанной таблицы
    #cv2.imwrite(output_path, cropped_image)

    return cropped_image

###### Функция улучшения точности изображения `acc_img`

(не дает положительного эффекта)

In [ ]:
def acc_img():
    """
    Увеличение точности изображения
    """
    command1 = f'python inference_realesrgan.py --model_path RealESRGAN_x4plus.pth --input {app.cropped_path} '
    command2 = f'--output {app.IMAGE_DIR} --fp32'
    full_command = command1 + command2
    subprocess.run(full_command, shell=True)


###### Функция распознавания `img2table`

In [ ]:
# функция для распознавания таблиц с изображения
# pytesseract для текста + EasyOCR для нераспознанного и цифр
def img2table(img_path, output_path):
    
    def sort_contours_yx(cnts, y_thresh=5):
        # Создание списка ограничивающих прямоугольников для каждого контура
        boxes = [cv2.boundingRect(c) for c in cnts]

        # Сортировка контуров по оси y
        sorted_boxes = sorted(boxes, key=lambda b: b[1])

        sorted_cnts = [x for _, x in sorted(zip(boxes, cnts), key=lambda pair: pair[0][1])]

        # Инициализация первого значения y и списка для сортировки по x
        last_y = sorted_boxes[0][1]
        current_group = []
        final_cnts = []

        for (x, y, w, h), c in zip(sorted_boxes, sorted_cnts):
            if abs(y - last_y) > y_thresh:  # Проверка разницы с предыдущим y
                # Сортировка текущей группы по x и сохранение результатов
                current_group.sort(key=lambda b: b[0])
                # Добавление отсортированной группы в финальный список
                final_cnts += current_group
                current_group = []
            current_group.append(((x, y, w, h), c))
            last_y = y

        # Не забываем про последнюю группу
        if current_group:
            current_group.sort(key=lambda b: b[0][0])
            final_cnts += current_group

        # Разделение контуров и боксов обратно в два списка
        final_boxes, final_cnts = zip(*final_cnts)

        return final_cnts, final_boxes

    # Считывание изображения
    img = img_path
    cv2_image = imutils.resize(img, height=500)

    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    edged = cv2.Canny(img, 75, 200)

    contours = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    grabbed = imutils.grab_contours(contours)
    sortedContours = sorted(grabbed, key=cv2.contourArea, reverse=True)[:5]

    screenCnt = max(sortedContours, key=cv2.contourArea)

    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    try:
        ratio = cv2_image.shape[0] / 500.0
        pts = screenCnt.reshape(4, 2) * ratio

        rect = np.zeros((4, 2), dtype="float32")

        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]

        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]
        rect[3] = pts[np.argmax(diff)]

        (tl, tr, br, bl) = rect

        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))

        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))

        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype="float32")

        M = cv2.getPerspectiveTransform(rect, dst)
        raw_transformed = cv2.warpPerspective(img, M, (maxWidth, maxHeight))
        bin_transformed = cv2.warpPerspective(img_bin, M, (maxWidth, maxHeight))
    except:
        raw_transformed = img
        bin_transformed = img_bin

    # Ширина ядра как 100-я часть общей ширины
    kernel_len = np.array(img).shape[1] // 100

    # Определение вертикального и горизонтального ядер
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

    # Ядро размером 2x2
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Обнаружение вертикальных и горизонтальных линий
    image_1 = cv2.erode(bin_transformed, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    image_2 = cv2.erode(bin_transformed, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Объединение горизонтальных и вертикальных линий в новом изображении
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

    # Размывание и установление порогового значения
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    _, img_vh = cv2.threshold(img_vh, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Обнаружение контуров
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Преобразование tuple в list
    contours = list(contours)
    contours.reverse()
    contours, _ = sort_contours_yx(contours, 5)

    texts = []
    # Итерация по каждому контуру (ячейке)
    for contour in contours:

        # Получение координат ограничивающего прямоугольника вокруг контура
        x, y, w, h = cv2.boundingRect(contour)
        if 0.04 < w / raw_transformed.shape[1] < 0.35 and 0.04 < h / raw_transformed.shape[0] < 0.35:
            cell_image = raw_transformed[y:y + h, x:x + w]

            # Получение текста из ячейки с помощью pytesseract
            text = pytesseract.image_to_string(cell_image, lang='rus')
            # распознавание нераспознанных ячеек и ячеек с циырами с помощью easyocr
            if text == '' or re.search(r'\d', text):
                results = reader.readtext(cell_image)
                if len(results) > 0:
                    text = results[0][1]

            # Добавление текста в список
            texts.append(text.replace('|', '').replace('_', '').replace(',', '.').replace('—', '').replace('\n', ''))
    
    # Обнаружение ячеек с датами
    dates = []
    date_ids = []
    for i in range(len(texts)):
        date = dateparser.parse(texts[i], languages=['ru'], date_formats=['%d/%m/%Y'], \
            settings={'REQUIRE_PARTS': ['day', 'month', 'year']})
        if type(date) == datetime.datetime and date > datetime.datetime(2000, 1, 1) and date <= datetime.datetime.now()  \
            and i != len(texts) - 1 and i != len(texts) - 2:
            dates.append(date.strftime('%d.%m.%Y'))
            date_ids.append(i)
    # распределение остальных ячеек вслед за датами
    kinds = []
    types = []
    quantities = []
    df = pd.DataFrame(columns=['Дата донации', 'Класс крови', 'Тип донации', 'Количество'])
    if date_ids:
        for i in date_ids:
            if 'бв' in texts[i + 1] or 'б' in texts[i + 1] or 'в' in texts[i + 1]:
                types.append('Безвозмездно')
            elif 'платно' in texts[i + 1]:
                types.append('Платно')
                texts[i + 1].replace('платно', '')
            else:
                types.append('')
            
            if 'кр' in texts[i + 1] or 'к' in texts[i + 1] or 'р' in texts[i + 1]:
                kinds.append('Цельная кровь')
            elif 'пл' in texts[i + 1] or 'п' in texts[i + 1] or 'л' in texts[i + 1]:
                kinds.append('Плазма')
            elif 'ц' in texts[i + 1] or 'т' in texts[i + 1]:
                kinds.append('Тромбоциты')
            else:
                kinds.append('')

            if texts[i + 2] != '':
                quantities.append(re.sub(r"\D", "", texts[i + 2]))
            else:
                quantities.append('')
        df['Дата донации'] = dates
        df['Класс крови'] = kinds
        df['Тип донации'] = types
        df['Количество'] = quantities

    # Сохранение результата распознавания
    #df.to_csv(output_path, index=False)

    return df

###### Функция сравнения результатов `compare_tables`

In [ ]:
# функция сравнения результатов распознавания с реальными значениями
# df_test - датасет проверочными значениями
# df - датасет с результатами распознавания
# df_rezult - датасет с корректно распознанными записями

def compare_tables(df_test, df):
    # унификация названий столбцов
    df.columns = ['Дата донации', 'Класс крови', 'Тип донации', 'Количество']
    # создание пустого датасета с корректно распознанными записями
    df_rezult = pd.DataFrame(columns=['Дата донации', 'Класс крови', 'Тип донации', 'Количество'])
    count_string = 0
    for i in range(df.shape[0]):
        for j in range(df_test.shape[0]):
            if (df.iloc[i,0:3] == df_test.iloc[j,0:3]).all():
                count_string += 1
                df_rezult = pd.concat([df_rezult, df.iloc[i,:].to_frame().T], ignore_index=True)
    return df_rezult, count_string

##### Основная программа

In [ ]:
# активация модели обнаружения таблиц
model = YOLO('keremberke/yolov8m-table-extraction')
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # maximum number of detections per image

In [ ]:
# активация pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [ ]:
# активация easyocr
reader = easyocr.Reader(['ru'])

07/25/2023 09:01:49 - WARNING - easyocr.easyocr -   CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
d:\Education\Data Science\Master\OCR\env\lib\site-packages\torchvision\models\_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(
d:\Education\Data Science\Master\OCR\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Education\Data Science\Master\OCR\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
%%time
# формирование списков изображений и тестовых csv, находящихся в подпапке test
path_file = r'test\\'
files_jpg = []
files_csv = []
for file in sorted(os.listdir(path_file)):
    if file.endswith('.jpg'):
        files_jpg.append(file)
    elif file.endswith('.csv'):
        files_csv.append(file)

# удаление лишних csv со словом rezult и лишних jpg со словом tabl в названии
files_jpg = [i for i in files_jpg if i.count('tabl') == 0]
files_csv = [i for i in files_csv if i.count('rezult') == 0]

accuracy = []

for i in range(len(files_jpg)):
    print(f'{i+1}. Результат распознавания файла {files_jpg[i]} (проверочный файл {files_csv[i]}):')
    # засекаем время
    start_time = time.time()
    # вызов функции вырезания таблицы
    tabl = crop_img(path_file+files_jpg[i], path_file+files_jpg[i][:-4]+'_tabl.jpg')
    # вызов функции распознавания
    df = img2table(tabl, path_file+files_jpg[i][:-4]+'_rezult.csv')
    # формирование проверочного датасета из csv-файла
    df_test = pd.read_csv(path_file+files_csv[i], usecols=['Дата донации', 'Класс крови', 'Тип донации'])[['Дата донации', 'Класс крови', 'Тип донации']]
    # вызов функции сравнения результата распознавания с проверочным датасетом
    df_rezult, count_row = compare_tables(df_test, df)
    accuracy.append(count_row)
    # подсчет времени выполнения
    time_fit = time.time() - start_time
    # отображение распознанной информации
    display(df)
    print(f'Корректно распознанные записи {count_row} шт. ({(count_row *100 / df_test.shape[0]):0.2f}%):')
    display(df_rezult.sort_values(by='Дата донации').reset_index(drop=1))
    print(f'Время распознавания файла {files_jpg[i]}: {time_fit:0.1f} сек. ({time.strftime("%H:%M:%S", time.gmtime(time_fit))})\n')
    
print(f'\nОбщий средний процент корректно распознанных записей во всех 15 справках {np.average(accuracy):0.2f}%.')

1. Результат распознавания файла 141899 .jpg (проверочный файл 141899.csv):


0: 640x448 1 bordered, 2549.8ms
Speed: 34.2ms preprocess, 2549.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


,Дата донации,Класс крови,Тип донации,Количество
0,02.02.2019,,,21062019
1,05.06.2020,,,26102020
2,26.08.2020,,,16112029
3,05.05.2022,,,1006208
4,27.03.2020,Плазма,Безвозмездно,600
5,10.07.2020,Плазма,Безвозмездно,


Корректно распознанные записи 2 шт. (6.90%):


,Дата донации,Класс крови,Тип донации,Количество
0,10.07.2020,Плазма,Безвозмездно,
1,27.03.2020,Плазма,Безвозмездно,600


...

0: 640x384 1 bordered, 3292.0ms
Speed: 76.2ms preprocess, 3292.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


,Дата донации,Класс крови,Тип донации,Количество
0,21.09.2017,Цельная кровь,,550
1,25.07.2022,Тромбоциты,,
2,25.01.2016,Плазма,Безвозмездно,192
3,03.12.2022,,,


Корректно распознанные записи 0 шт. (0.00%):


,Дата донации,Класс крови,Тип донации,Количество


Время распознавания файла 256838.jpg: 133.4 сек. (00:02:13)


Общий средний процент корректно распознанных записей во всех 15 справках 8.67%.
CPU times: total: 1h 17min 40s
Wall time: 21min 41s


#### 2.2.5 Создание микросервиса для включения в систему Зказчика

Для включения в систему Заказчика разработан микросервис с использованием библиотеки `fastapi`. Сервис позволяет выбрать на диске и распознать файл с изображением справки. В случае плохого распознавания, пользователь может применить дополнительную предобработку изображения нажатием соответствующей кнопки и попробовать улучшить результат. Дополнительная предобработка осуществляется с помощью предобученной модели `RealESRGAN`.